In [97]:
import pandas as pd

from constants.abs_paths import AbsDirPath


df = pd.read_csv(AbsDirPath.ROOT / "processing_pipeline" / "analysis" / "param_tuning.csv")

## Processing metrics per items

In [98]:
from processing_pipeline.utilities.data_transformation import load_all_files

df_items = load_all_files(AbsDirPath.PARAMETER_TUNING_DIR)
df_items.shape

Loaded C:\Users\Dmitrijs\Documents\myDocs\masters\courses\thesis\code\data\keywords\parameter_tuning\allenai.scispacy.v0.5.5.CODE_COMMENT.first_100.parquet
Loaded C:\Users\Dmitrijs\Documents\myDocs\masters\courses\thesis\code\data\keywords\parameter_tuning\allenai.scispacy.v0.5.5.DOCS.first_100.parquet
Loaded C:\Users\Dmitrijs\Documents\myDocs\masters\courses\thesis\code\data\keywords\parameter_tuning\allenai.scispacy.v0.5.5.ISSUE.first_100.parquet
Loaded C:\Users\Dmitrijs\Documents\myDocs\masters\courses\thesis\code\data\keywords\parameter_tuning\allenai.scispacy.v0.5.5.ISSUE_COMMENT.first_100.parquet
Loaded C:\Users\Dmitrijs\Documents\myDocs\masters\courses\thesis\code\data\keywords\parameter_tuning\allenai.scispacy.v0.5.5.RELEASES.first_100.parquet
Loaded C:\Users\Dmitrijs\Documents\myDocs\masters\courses\thesis\code\data\keywords\parameter_tuning\allenai.scispacy.v0.5.5.WIKI.first_100.parquet
Loaded C:\Users\Dmitrijs\Documents\myDocs\masters\courses\thesis\code\data\keywords\parame

(1245, 14)

In [99]:
df_items.head()

,quality_attribute,keyword,matched_word,match_idx,sentence,source,filename,author,repo,version,wiki,url,text_hash,similar_count
0,availability,down,download,102.0,""""""". Given something that might be a URL (or m...",CODE_COMMENT,scispacy/file_cache.py,allenai,scispacy,v0.5.5,https://allenai.github.io/scispacy/,https://github.com/allenai/scispacy/tree/v0.5....,24a76770099171176eadd8bfa96cca0d,1
1,availability,down,downloading,33.0,"# URL, so get it from the cache (downloading i...",CODE_COMMENT,scispacy/file_cache.py,allenai,scispacy,v0.5.5,https://allenai.github.io/scispacy/,https://github.com/allenai/scispacy/tree/v0.5....,61a731adbe71e0403ca12a95d1890c48,1
2,availability,consist,consists,580.0,""""""". A spacy pipeline component which identifi...",CODE_COMMENT,scispacy/linking.py,allenai,scispacy,v0.5.5,https://allenai.github.io/scispacy/,https://github.com/allenai/scispacy/tree/v0.5....,9f212991e788b39a27ba63c9b6f825cd,1
3,availability,down,download,2.0,"# Download to temporary file, then copy to cac...",CODE_COMMENT,scispacy/file_cache.py,allenai,scispacy,v0.5.5,https://allenai.github.io/scispacy/,https://github.com/allenai/scispacy/tree/v0.5....,cc907e53d7b1b288b9feccbd5ad06d83,2
4,availability,avail,available,94.0,""""""". A candidate generator for entity linking ...",CODE_COMMENT,scispacy/candidate_generation.py,allenai,scispacy,v0.5.5,https://allenai.github.io/scispacy/,https://github.com/allenai/scispacy/tree/v0.5....,cce7b6bf5d4752be2dfd20cddee72b30,1


In [100]:
n_items = df_items.shape[0]
df["time_per_item"] = df.time / n_items
df.head()

,n_threads,n_batches,model_name,time,time_per_item
0,1,5,deepseek-r1:1.5b,710.782597,0.570910
1,1,5,deepseek-r1:7b,509.096897,0.408913
2,1,5,deepseek-r1:8b,682.476232,0.548174
3,1,10,deepseek-r1:1.5b,391.115945,0.314149
4,1,10,deepseek-r1:7b,493.770622,0.396603


In [101]:
min_value = df.time_per_item.min()
df["time_per_item_delta"] = df.time_per_item - min_value
df["time_per_item_delta"] = df.time_per_item_delta.astype("float").round(5)
df["time_per_item_delta"] = df.time_per_item_delta.astype("str").apply(lambda x: x if x == "0" else "+" + x)
df.head()

,n_threads,n_batches,model_name,time,time_per_item,time_per_item_delta
0,1,5,deepseek-r1:1.5b,710.782597,0.570910,+0.26506
1,1,5,deepseek-r1:7b,509.096897,0.408913,+0.10306
2,1,5,deepseek-r1:8b,682.476232,0.548174,+0.24232
3,1,10,deepseek-r1:1.5b,391.115945,0.314149,+0.0083
4,1,10,deepseek-r1:7b,493.770622,0.396603,+0.09075


In [102]:
items_filter_rate_per_stage = .75
est_nitems_s0 = 524909
est_nitems_s1 = 1241367 * items_filter_rate_per_stage
est_nitems_s2 = 524909 * (items_filter_rate_per_stage ** 2)
est_nitems_s3 = 524909 * (items_filter_rate_per_stage ** 3)

est_nitems_s0, est_nitems_s1, est_nitems_s2, est_nitems_s3

(524909, 931025.25, 295261.3125, 221445.984375)

In [103]:
seconds_per_day = 24 * 60 * 60
df["est_time_s0_days"] = est_nitems_s0 * df.time_per_item / seconds_per_day
df["est_time_s1_days"] = est_nitems_s1 * df.time_per_item / seconds_per_day
df["est_time_s2_days"] = est_nitems_s2 * df.time_per_item / seconds_per_day
df["est_time_s3_days"] = est_nitems_s3 * df.time_per_item / seconds_per_day

df["est_time_s0_days"] = df.est_time_s0_days.astype("float").round(2)
df["est_time_s1_days"] = df.est_time_s1_days.astype("float").round(2)
df["est_time_s2_days"] = df.est_time_s2_days.astype("float").round(2)
df["est_time_s3_days"] = df.est_time_s3_days.astype("float").round(2)

df["est_time_total_days"] = df[["est_time_s0_days", "est_time_s1_days", "est_time_s2_days", "est_time_s3_days"]].sum(axis=1)
two_server_boost = .75
df["est_time_total_2machines_days"] = df.est_time_total_days * two_server_boost

## Analyze results

In [104]:
df.head()

,n_threads,n_batches,model_name,time,time_per_item,time_per_item_delta,est_time_s0_days,est_time_s1_days,est_time_s2_days,est_time_s3_days,est_time_total_days,est_time_total_2machines_days
0,1,5,deepseek-r1:1.5b,710.782597,0.570910,+0.26506,3.47,6.15,1.95,1.46,13.03,9.7725
1,1,5,deepseek-r1:7b,509.096897,0.408913,+0.10306,2.48,4.41,1.40,1.05,9.34,7.0050
2,1,5,deepseek-r1:8b,682.476232,0.548174,+0.24232,3.33,5.91,1.87,1.40,12.51,9.3825
3,1,10,deepseek-r1:1.5b,391.115945,0.314149,+0.0083,1.91,3.39,1.07,0.81,7.18,5.3850
4,1,10,deepseek-r1:7b,493.770622,0.396603,+0.09075,2.41,4.27,1.36,1.02,9.06,6.7950


In [105]:
df.sort_values("time")

,n_threads,n_batches,model_name,time,time_per_item,time_per_item_delta,est_time_s0_days,est_time_s1_days,est_time_s2_days,est_time_s3_days,est_time_total_days,est_time_total_2machines_days
45,15,5,deepseek-r1:1.5b,380.782035,0.305849,+0.0,1.86,3.30,1.05,0.78,6.99,5.2425
18,5,10,deepseek-r1:1.5b,383.611733,0.308122,+0.00227,1.87,3.32,1.05,0.79,7.03,5.2725
15,5,5,deepseek-r1:1.5b,384.947358,0.309195,+0.00335,1.88,3.33,1.06,0.79,7.06,5.2950
54,15,20,deepseek-r1:1.5b,385.159000,0.309365,+0.00352,1.88,3.33,1.06,0.79,7.06,5.2950
21,5,15,deepseek-r1:1.5b,385.687966,0.309790,+0.00394,1.88,3.34,1.06,0.79,7.07,5.3025
9,1,20,deepseek-r1:1.5b,385.888852,0.309951,+0.0041,1.88,3.34,1.06,0.79,7.07,5.3025
42,10,50,deepseek-r1:1.5b,386.105786,0.310125,+0.00428,1.88,3.34,1.06,0.79,7.07,5.3025
51,15,15,deepseek-r1:1.5b,386.284472,0.310269,+0.00442,1.88,3.34,1.06,0.80,7.08,5.3100
36,10,15,deepseek-r1:1.5b,386.559649,0.310490,+0.00464,1.89,3.35,1.06,0.80,7.10,5.3250
27,5,50,deepseek-r1:1.5b,386.718911,0.310618,+0.00477,1.89,3.35,1.06,0.80,7.10,5.3250


In [106]:
df.pivot_table(index=["model_name"], columns=["n_threads" , "n_batches"], values="time", aggfunc="mean")

n_threads                 1                                                   \
n_batches                 5           10          15          20          50   
model_name                                                                     
deepseek-r1:1.5b  710.782597  391.115945  394.316178  385.888852  388.833056   
deepseek-r1:7b    509.096897  493.770622  494.078939  495.871160  493.855564   
deepseek-r1:8b    682.476232  648.983599  646.518526  637.916200  633.896038   

n_threads                 5                                                   \
n_batches                 5           10          15          20          50   
model_name                                                                     
deepseek-r1:1.5b  384.947358  383.611733  385.687966  694.394862  386.718911   
deepseek-r1:7b    487.743229  483.860421  486.187780  489.986789  483.858473   
deepseek-r1:8b    627.499402  627.763133  638.157547  632.273979  628.706981   

n_threads                 10                                                  \
n_batches                 5           10          15          20          50   
model_name                                                                     
deepseek-r1:1.5b  698.375105  388.969251  386.559649  392.152330  386.105786   
deepseek-r1:7b    479.253000  491.678863  491.311730  486.066361  490.301308   
deepseek-r1:8b    629.357629  636.225479  641.566748  635.752086  623.457960   

n_threads                 15                                                  
n_batches                 5           10          15          20          50  
model_name                                                                    
deepseek-r1:1.5b  380.782035  678.423196  386.284472  385.159000  388.794956  
deepseek-r1:7b    487.333505  491.100365  489.212522  487.826407  492.429089  
deepseek-r1:8b    624.790384  631.706247  626.656877  633.795894  634.667988

In [107]:
df.pivot_table(index=["model_name"], columns=["n_batches"], values="time", aggfunc="mean")

n_batches,5,10,15,20,50
model_name,,,,,
deepseek-r1:1.5b,543.721774,460.530031,388.212066,464.398761,387.613177
deepseek-r1:7b,490.856658,490.102568,490.197743,489.937679,490.111109
deepseek-r1:8b,641.030912,636.169614,638.224925,634.934540,630.182242


In [108]:
df.pivot_table(index=["model_name"], columns=["n_threads"], values="time", aggfunc="mean")

n_threads,1,5,10,15
model_name,,,,
deepseek-r1:1.5b,454.187326,447.072166,450.432424,443.888732
deepseek-r1:7b,497.334636,486.327338,487.722252,489.580378
deepseek-r1:8b,649.958119,630.880208,633.271980,630.323478


In [109]:
df.groupby(["model_name", "n_threads", "n_batches"]).time.agg("mean")

model_name        n_threads  n_batches
deepseek-r1:1.5b  1          5            710.782597
                             10           391.115945
                             15           394.316178
                             20           385.888852
                             50           388.833056
                  5          5            384.947358
                             10           383.611733
                             15           385.687966
                             20           694.394862
                             50           386.718911
                  10         5            698.375105
                             10           388.969251
                             15           386.559649
                             20           392.152330
                             50           386.105786
                  15         5            380.782035
                             10           678.423196
                             15           386.284472
       

In [110]:
df.loc[df.groupby(["model_name", "n_threads"]).time.idxmin()].groupby(["model_name", "n_threads", "n_batches"]).time.agg("mean")

model_name        n_threads  n_batches
deepseek-r1:1.5b  1          20           385.888852
                  5          10           383.611733
                  10         50           386.105786
                  15         5            380.782035
deepseek-r1:7b    1          10           493.770622
                  5          50           483.858473
                  10         5            479.253000
                  15         5            487.333505
deepseek-r1:8b    1          50           633.896038
                  5          5            627.499402
                  10         50           623.457960
                  15         5            624.790384
Name: time, dtype: float64

In [111]:
df.loc[df.groupby(["model_name"]).time.idxmin()]

,n_threads,n_batches,model_name,time,time_per_item,time_per_item_delta,est_time_s0_days,est_time_s1_days,est_time_s2_days,est_time_s3_days,est_time_total_days,est_time_total_2machines_days
45,15,5,deepseek-r1:1.5b,380.782035,0.305849,+0.0,1.86,3.30,1.05,0.78,6.99,5.2425
31,10,5,deepseek-r1:7b,479.253000,0.384942,+0.07909,2.34,4.15,1.32,0.99,8.80,6.6000
44,10,50,deepseek-r1:8b,623.457960,0.500769,+0.19492,3.04,5.40,1.71,1.28,11.43,8.5725
